In [314]:
import numpy as np

In [315]:
a = 1
U0 = 1
E0 = -0.95
precision = 10e-9

In [316]:
def f(E):
	return (1 / np.tan(np.sqrt(2 * a ** 2 * U0 * (1 + E / U0)))) - np.sqrt(-U0 / E - 1)

def df(E):
	return - (a ** 2) / (
			(np.sin(np.sqrt(2 * a * 2 * U0 * (1 + E / U0))) ** 2) * np.sqrt(2 * a ** 2 * U0 * (1 + E / U0))) - (
				   U0 / 2) / (E ** 2 * ((-U0 / E) - 1))

In [317]:
def dichotomy(a, b, error=precision):
	middle = (a + b) / 2
	curError = (b - a) / 2
	if curError < error:
		return middle
	return dichotomy(a, middle) if f(middle) * f(a) <= 0 else dichotomy(middle, b)

x_0 = -1
x_1 = -0
dichotomyRoot = dichotomy(x_0, x_1)

# в качестве начального приближения возьмем корень, полученный методом дихотомии
x_initial = dichotomyRoot
Lambda = 1 / df(x_initial) + 0.001

def fixedPointIteration(x, x0=x_initial, error=precision):
	value = x - Lambda * f(x)
	curError = np.abs(x0 - x)
	return fixedPointIteration(value) if curError > error else x

fixPointIterRoot = fixedPointIteration(E0)

def Newton(x, error=precision):
	value = x - f(x) / df(x)
	curError = np.abs(fixPointIterRoot - x)
	return Newton(value) if curError > error else x

newtonRoot = Newton(E0)

C:\Users\константин\AppData\Local\Temp\ipykernel_17448\3627897899.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  return (1 / np.tan(np.sqrt(2 * a ** 2 * U0 * (1 + E / U0)))) - np.sqrt(-U0 / E - 1)


In [318]:
print('NewtonMethod - ' + str(newtonRoot))
print('f({}) = {}'.format(newtonRoot, f(newtonRoot)))
print('FixedPointIterationsMethod - ' + str(fixPointIterRoot))
print('f({}) = {}'.format(fixPointIterRoot, f(fixPointIterRoot)))
print('DichotomyMethod - ' + str(dichotomyRoot))
print('f({}) = {}'.format(dichotomyRoot, f(dichotomyRoot)))

NewtonMethod - -0.6038978305858146
f(-0.6038978305858146) = -1.164625840210931e-08
FixedPointIterationsMethod - -0.6038978353098972
f(-0.6038978353098972) = 5.139872416570768e-09
DichotomyMethod - -0.6038978323340416
f(-0.6038978323340416) = -5.4342654820516145e-09
